In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, random_split
import numpy as np
import pandas as pd
import math
import torch.nn.functional as F
import sys
import csv
from torch.nn.functional import normalize

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Network Construction

Construct multilayer neural network for the pain prediction dataset with input dimension of 22.

In [2]:
class InferenceBlock(nn.Module):
    def __init__(self, input_units, d_theta, output_units):
        """
        :param d_theta: dimensionality of the intermediate hidden layers.
        :param output_units: dimensionality of the output.
        :return: batch of outputs.
        """
        super(InferenceBlock, self).__init__()
        self.module = nn.Sequential(
            nn.Linear(input_units, d_theta, bias=True),
            nn.ELU(inplace=True),
            nn.Linear(input_units, d_theta, bias=True),
            nn.ELU(inplace=True),
            nn.Linear(d_theta, output_units, bias=True)
        )

    def forward(self, inps):
        out = self.module(inps)
        return out

class Amortized(nn.Module):
    def __init__(self, input_units=400, d_theta=400, output_units=400):
        super(Amortized, self).__init__()
        self.output_units = output_units
        self.weight_mean         = InferenceBlock(input_units, d_theta, output_units)
        self.weight_log_variance = InferenceBlock(input_units, d_theta, output_units)

    def forward(self, inps):
        weight_mean         = self.weight_mean(inps)
        weight_log_variance = self.weight_log_variance(inps)
        return weight_mean, weight_log_variance


class MLP(nn.Module):
    """
    Two layer MLP for pain prediction dataset benchmarks.
    """
    def __init__(self, hiddens, config):
        super(MLP, self).__init__()
        self.W1 = nn.Linear(22, hiddens)
        self.relu = nn.ReLU(inplace=True)
        self.dropout_1 = nn.Dropout(p=config['dropout'])
        self.W2 = nn.Linear(hiddens, hiddens)
        self.dropout_2 = nn.Dropout(p=config['dropout'])
        # self.W3 = nn.Linear(hiddens, 10)
        self.post  = Amortized(hiddens, hiddens, hiddens)
        self.prior = Amortized(hiddens, hiddens, hiddens)

    def forward(self, x, task_id=None, post = False, predefined = False):
        x = x.view(-1, 22)
        out = self.W1(x)
        out = self.relu(out)
        out = self.dropout_1(out)
        out = self.W2(out)
        out = self.relu(out)
        out = self.dropout_2(out)
        out_features = self.normalize(out)
        out_mean = torch.mean(out_features, dim=0, keepdim=True)
        if post:
            mu, logvar = self.post(out_mean)
        else:
            mu, logvar = self.prior(out_mean)
        return out_features, mu, logvar

    def normalize(self, x):
        max_val = x.max()
        min_val = x.min()
        return (x - min_val) / (max_val - min_val)
    
def get_pain_prediction(num_tasks):

    """
    Returns the dataset loaders for all tasks of pain prediction dataset
    """

    datasets = {}
    data_folder = r"C:\Users\ronny\Documents\GitHub\MetaPain\Data\biosignals_filtered_Processed\ecg"
    files = os.listdir(data_folder)
    for task_id in range(1,num_tasks+1):

#         with open(os.path.join(data_folder, files[task_id-1]),'r') as f:
        with open(data_folder+'/'+ files[task_id-1],'r') as f:

            pickle_in = list(csv.reader(f, delimiter=','))
            feature,target = np.array(pickle_in)[:,:-1].astype(float),np.array(pickle_in)[:,-1]
            target = np.unique(target, return_inverse=True)[1].astype(float)
            feature = normalize(torch.Tensor(feature),dim=0)
            dataset = TensorDataset(feature,torch.Tensor(target))
            train_dataset, test_dataset = random_split(dataset, [70, 30], generator=torch.Generator().manual_seed(42))
            datasets[task_id] = {'train': train_dataset, 'test': test_dataset}

    return datasets
    
def get_benchmark_model(hiddens, dropout):
    """
    Return the corresponding PyTorch model for experiment
    :param args:
    :return:
    """
    return MLP(hiddens, {'dropout': dropout}).to(DEVICE)

def get_benchmark_dataset(dataset):
    """
    Returns the benchmark loader which could be either of these:
    get_split_cifar100_tasks, get_permuted_mnist_tasks, or get_rotated_mnist_tasks

    :param args:
    :return: a function which when called, returns all tasks
    """
    if dataset == "pain_prediction":
        return get_pain_prediction
    else:
        raise ValueError("this function can only get pain_prediction dataset")

In [3]:
EXPERIMENT_DIRECTORY = r"C:\Users\ronny\Documents\GitHub\MetaPain\simulation"

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def iterate_data(loader):
    xs,ys = [], []
    for x,y in loader:
        xs.append(x)
        ys.append(y)
    xs = torch.cat(xs, dim=0).squeeze(1)
    ys = torch.cat(ys, dim=0)
    return xs, ys


def sample(mu, logvar, L, device):
    shape = (L, ) + mu.size()
    eps   = torch.randn(shape).to(device)
    w     = mu.unsqueeze(0) + eps * logvar.exp().sqrt().unsqueeze(0)
    return w

def rand_features(bases, features, bias):
    # tf.random_normal()
    return math.sqrt(2/bias.shape[0]) * torch.cos(torch.matmul(bases, features) + bias)

def dotp_kernel(feature1, feature2):
    return torch.matmul(feature1, feature2)

def rbf_kernel(feature1, feature2):
    res = torch.exp(-0.25 * torch.norm(feature1.unsqueeze(1)-feature2, dim=2, p=1))
    return res

def linear(feature1, feature2):
    return feature1 @ feature2.T

def poly(feature1, feature2):
    return (torch.matmul(feature1, feature2.T) + 1).pow(3)

def kl_div(m, log_v, m0, log_v0):

    v = log_v.exp()
    v0 = log_v0.exp()

    dout, din = m.shape
    const_term = -0.5 * dout * din

    log_std_diff = 0.5 * torch.sum(torch.log(v0) - torch.log(v))
    mu_diff_term = 0.5 * torch.sum((v + (m0-m)**2) / v0)
    kl = const_term + log_std_diff + mu_diff_term
    return kl

def cosine_dist(a, b):
    # sqrt(<a, b>) / (sqrt(<a, a>), sqrt(<b, b>))
    a = a.view(1, -1)
    b = b.view(1, -1)
    normalize_a = F.normalize(a, dim=-1)# %
    normalize_b = F.normalize(b, dim=-1)
    return torch.sqrt(torch.sum(torch.multiply(normalize_a, normalize_b)))


def get_train_loader(dataset, batch_size):
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=8, pin_memory=True, shuffle=True)
    return train_loader


def get_test_loader(dataset):
    test_loader  = torch.utils.data.DataLoader(dataset,  batch_size=5, shuffle=False, num_workers=3, pin_memory=True)
    return test_loader

def log_metrics(metrics, time, prev_task_id, current_task_id, acc_db, loss_db):
    """
    Log accuracy and loss at different times of training
    """
    print('epoch {}, task:{}, metrics: {}'.format(time, prev_task_id, metrics))
    # log to db
    acc = metrics['accuracy']
    loss = metrics['loss']
    loss_db[prev_task_id][current_task_id-1]= loss
    acc_db[prev_task_id][current_task_id-1] = acc
    return acc_db, loss_db


def init_experiment(tasks):
    print('------------------- Experiment started -----------------')
    # print(f"Parameters:\n  seed={args.seed}\n  benchmark={args.dataset}\n  num_tasks={args.tasks}\n  "+
    # 	  f"epochs_per_task={args.epochs_per_task}\n  batch_size={args.batch_size}\n  "+
    # 	  f"learning_rate={args.lr}\n  learning rate decay(gamma)={args.gamma}\n  dropout prob={args.dropout}\n")

    # # 1. setup seed for reproducibility
    # random.seed(args.seed)
    # np.random.seed(args.seed)
    # torch.manual_seed(args.seed)
    # torch.cuda.manual_seed(args.seed)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = False
    # # torch.backends.cudnn.enabled = False

    # # 2. create directory to save results
    # Path(EXPERIMENT_DIRECTORY).mkdir(parents=True, exist_ok=True)
    # print("The results will be saved in {}\n".format(EXPERIMENT_DIRECTORY))

    # 3. create data structures to store metrics
    # loss_db = {t: [0 for i in range(args.tasks*args.epochs_per_task)] for t in range(1, args.tasks+1)}
    # acc_db =  {t: [0 for i in range(args.tasks*args.epochs_per_task)] for t in range(1, args.tasks+1)}
    loss_db = {t: [0 for i in range(tasks)] for t in range(1, tasks+1)}
    acc_db =  {t: [0 for i in range(tasks)] for t in range(1, tasks+1)}
    return acc_db, loss_db


def end_experiment(acc_db,d_rn_f):

    # 1. save all metrics into csv file
    acc_df = pd.DataFrame(acc_db)
    acc_df.to_csv(EXPERIMENT_DIRECTORY+'/accs.csv')


    # 2. calculate average accuracy and forgetting (c.f. ``evaluation`` section in our paper)
    score = np.mean([acc_db[i][-1] for i in acc_db.keys()])

    original_stdout = sys.stdout
    with open(EXPERIMENT_DIRECTORY+'/final_result'+str(d_rn_f)+'.txt', 'w') as f:
        sys.stdout = f
        print('average accuracy = {}'.format(score))
        sys.stdout = original_stdout
    print('average accuracy = {}'.format(score))
    print()
    print('------------------- Experiment ended -----------------')

def coreset_selection(dataset, coreset_size, dataset_name):
    # Uniformly sample and select data per each class from (x_train, y_train) and add to current coreset (x_coreset, y_coreset)
    if 'mnist' in dataset_name:
        targets = dataset.targets
        sample_per_each_class = coreset_size // len(targets.unique())
    else:
        loader = get_test_loader(dataset)
        inputs, targets = iterate_data(loader)
        sample_per_each_class = coreset_size


    idx = []
    for cl in targets.unique():
        cl = cl.item()
        cls_idx = torch.where(targets == cl)[0]
        if len(cls_idx) == 0:
            continue
        cls_idx = cls_idx[torch.randperm(len(cls_idx))[:sample_per_each_class]]
        idx = idx + cls_idx.tolist()

    remained_item = list(set(torch.arange(0, len(targets)).tolist()) - set(idx))
    if 'mnist' in dataset_name:
        train_dataset    = torch.utils.data.dataset.Subset(dataset, remained_item)
        coreset_dataset  = torch.utils.data.dataset.Subset(dataset, idx)
    else:
        train_dataset    = torch.utils.data.dataset.TensorDataset(inputs[remained_item], targets[remained_item])
        coreset_dataset  = torch.utils.data.dataset.TensorDataset(inputs[idx], targets[idx])
    return train_dataset, coreset_dataset

In [4]:
def train_single_epoch(net, optimizer, loader, coreset_loader, criterion, bias, lmd, gma, bta, d_rn_f, tau, task_id=None):
    """
    Train the model for a single epoch

    :param net:
    :param optimizer:
    :param loader:
    :param criterion:
    :param task_id:
    :return:
    """
    net = net.to(DEVICE)
    net.train()

    coreset_input, coreset_target = iterate_data(coreset_loader)
    coreset_input  = coreset_input.to(DEVICE)
#     coreset_target = torch.nn.functional.one_hot(coreset_target, num_classes=5)
    coreset_target = coreset_target.to(DEVICE).float()

    task_acc = AverageMeter()
    task_loss = AverageMeter()

    # import pdb; pdb.set_trace()
    for _, (data, target) in enumerate(loader):
        data = data.to(DEVICE)
        target = target.to(DEVICE)
        batch_size = data.shape[0]
        optimizer.zero_grad()

        if task_id:
            features_train,   p_mu, p_log_var = net(data, task_id, post = False, predefined=True)
            features_coreset, r_mu, r_log_var  = net(coreset_input, task_id, post = True)
        else:
            features_train,   p_mu, p_log_var = net(data,          post = False, predefined=True)
            features_coreset, r_mu, r_log_var  = net(coreset_input, post = True)

        # compute_bases
        rs = sample(r_mu, r_log_var, d_rn_f, DEVICE).squeeze()


        # Here we map the coreset data into random fourier space. 
        x_supp_phi_t = rand_features(rs, torch.transpose(features_coreset, 1, 0), bias)  # (d_rn_f , number of coreset samples)
        # in the random fourier space we compute the the kernel matrix, the dot product between mapped samples
        support_kernel = dotp_kernel(torch.transpose(x_supp_phi_t, 1, 0), x_supp_phi_t)  # (number of coreset samples , number of coreset samples)

        # closed-form solution with trainable lmd
        # alpha = torch.matmul(torch.inverse(support_kernel + (lmd_abs + 0.01) * torch.eye(support_kernel[0])), target)
        # (number of coreset samples, number of classes)
        alpha = torch.matmul(torch.inverse(support_kernel + (torch.abs(lmd[task_id-1]) + 0.01) * torch.eye(support_kernel.shape[0]).to(DEVICE)), coreset_target) 

        # Here we map the train data into random fourier space. 
        x_que_phi_t = rand_features(rs, torch.transpose(features_train, 1, 0), bias)

        # Compute cross kernel
        cross_kernel = dotp_kernel(torch.transpose(x_supp_phi_t, 1, 0), x_que_phi_t) # (number of coreset samples, number of train samples)

        pred = gma[task_id-1] * torch.matmul(cross_kernel.T, alpha) + bta[task_id-1]

        kl_loss = kl_div(r_mu, r_log_var, p_mu, p_log_var)

        loss = criterion(pred, target) + tau * kl_loss
        task_loss.update(loss.item())

        acc = (pred.argmax(dim=-1) == target).float().mean()
        task_acc.update(acc.item())


        loss.backward()
        optimizer.step()

    print(task_acc.avg, task_loss.avg)
    return net

In [5]:
def eval_single_epoch(net, loader, coreset_loader, criterion, bias, lmd, gma, bta, d_rn_f, tau, task_id=None):
    """
    Evaluate the model for single epoch

    :param net:
    :param loader:
    :param criterion:
    :param task_id:
    :return:
    """

    net = net.to(DEVICE)
    net.eval()

    coreset_input, coreset_target = iterate_data(coreset_loader)
    coreset_input  = coreset_input.to(DEVICE)
#     coreset_target = torch.nn.functional.one_hot(coreset_target, num_classes=5)
    coreset_target = coreset_target.to(DEVICE).float()

    test_loss = AverageMeter()
    correct   = AverageMeter()

    with torch.no_grad():
        for data, target in loader:
            data = data.to(DEVICE)
            target = target.to(DEVICE)
            if task_id:
                features_test,   p_mu, p_log_var = net(data, task_id, post = False)
                features_coreset, r_mu, r_log_var  = net(coreset_input, task_id, post = True)
            else:
                features_test,   p_mu, p_log_var = net(data,          post = False)
                features_coreset, r_mu, r_log_var  = net(coreset_input, post = True)

            # compute_bases
            rs = sample(r_mu, r_log_var, d_rn_f, DEVICE).squeeze()

            x_supp_phi_t = rand_features(rs, torch.transpose(features_coreset, 1, 0), bias)  

            support_kernel = dotp_kernel(torch.transpose(x_supp_phi_t, 1, 0), x_supp_phi_t)  

            # closed-form solution with trainable lmd
            alpha = torch.matmul(torch.inverse(support_kernel + (torch.abs(lmd[task_id-1]) + 0.01) * torch.eye(support_kernel.shape[0]).to(DEVICE)), coreset_target)
            x_que_phi_t = rand_features(rs, torch.transpose(features_test, 1, 0), bias)

            # Compute cross kernel
            cross_kernel = dotp_kernel(torch.transpose(x_supp_phi_t, 1, 0), x_que_phi_t)

            pred = gma[task_id-1] * torch.matmul(cross_kernel.T, alpha) + bta[task_id-1]
            # align loss
            
            loss = criterion(pred, target)

            test_loss.update(loss.item())

            acc = (pred.argmax(dim=-1) == target).float().mean()
            correct.update(acc.item())

    return {'accuracy': correct.avg, 'loss': test_loss.avg}

In [6]:
def run(num_tasks, hiddens, d_rn_f, epochs_per_task, batch_size, core_size, model_name, tau = 0.0001):
    """
    Run a single run of experiment.

    :param args: please see `utils.py` for arguments and options
    """
    

    # pdb.set_trace()
    acc_db, loss_db = init_experiment(num_tasks)
    print("Loading {} tasks for pain prediction".format(num_tasks))
    tasks = get_benchmark_dataset("pain_prediction")(num_tasks)
    print("loaded all tasks!")
    model = get_benchmark_model(hiddens, dropout=0.3)

    # criterion
    criterion = nn.MSELoss().to(DEVICE)
    bias = 2 * math.pi * torch.rand(d_rn_f, 1).to(DEVICE)
    time = 0

    # Learnable coefficient

    lmd   = torch.tensor([0.5      for _ in range(num_tasks)], requires_grad=True, device=DEVICE)
    gma   = torch.tensor([1.0      for _ in range(num_tasks)], requires_grad=True, device=DEVICE)
    bta   = torch.tensor([0.       for _ in range(num_tasks)], requires_grad=True, device=DEVICE)

    coreset_all = []
    # basis = []
    # rff = []
    # mu = []
    # log_var = []
    for current_task_id in range(1, num_tasks+1):
        print("================== TASK {} / {} =================".format(current_task_id, num_tasks))
        train_dataset = tasks[current_task_id]['train']
        train_dataset, coreset_dataset = coreset_selection(train_dataset, core_size, dataset_name = "pain_prediction")

        # import pdb; pdb.set_trace()
        # save coreset
        coreset_all.append(coreset_dataset)

        # uncomment for replay
        # coreset_dataset = torch.utils.data.dataset.ConcatDataset(coreset_all)
        # print(len(coreset_dataset))

        train_loader    = get_train_loader(train_dataset, batch_size)
        coreset_loader  = get_train_loader(coreset_dataset, batch_size)
        lr = max(0.1 * 0.5 ** (current_task_id), 0.00005) 
        for epoch in range(1, epochs_per_task+1):
            # 1. train and save
            params = list(model.parameters()) + [lmd, gma, bta]
            optimizer = torch.optim.SGD(params, lr=lr, momentum=0.8)  
            train_single_epoch(model, optimizer, train_loader, coreset_loader, criterion, bias, lmd, gma, bta, d_rn_f, tau, current_task_id)
            time += 1
            # import pdb; pdb.set_trace()
            # 2. evaluate on all tasks up to now, including the current task
            for prev_task_id in range(1, current_task_id+1):
                # 2.0. only evaluate once a task is finished
                if epoch == epochs_per_task:
                    model = model.to(DEVICE)
                    val_dataset = tasks[prev_task_id]['test']
                    coreset_dataset = coreset_all[prev_task_id-1]

                    val_loader      = get_test_loader(val_dataset)
                    coreset_loader  = get_test_loader(coreset_dataset)

                    # 2.1. compute accuracy and loss
                    metrics = eval_single_epoch(model, val_loader, coreset_loader, criterion, bias, lmd, gma, bta, d_rn_f, tau, current_task_id)
                    acc_db, loss_db = log_metrics(metrics, time, prev_task_id, current_task_id, acc_db, loss_db)

#                     # 2.3. save model parameters
#                     save_checkpoint(model, time)
        # basis, rff, mu, log_var = save_stats_one_task(model, coreset_all, bias, args, 1, basis, rff, mu, log_var)
    torch.save(model.state_dict(), EXPERIMENT_DIRECTORY+'/'+model_name)
    end_experiment(loss_db,d_rn_f)

In [7]:
run(num_tasks = 86, hiddens=64, d_rn_f = 32, epochs_per_task = 30, batch_size = 10, core_size = 6,model_name = 'MSE_32', tau = 1)

------------------- Experiment started -----------------
Loading 86 tasks for pain prediction
loaded all tasks!
================== TASK 1 / 86 =================
0.07500000111758709 3.8062079548835754
0.07500000111758709 2.7299112677574158
0.12500000186264515 2.1925975382328033
0.10000000335276127 2.7676831483840942
0.02500000037252903 2.1291127502918243
0.10000000149011612 2.373998701572418
0.05000000074505806 2.349003940820694
0.10000000149011612 2.177640587091446
0.2000000011175871 2.1970531940460205
0.15000000223517418 2.7152667343616486
0.07500000111758709 2.7258008122444153
0.10000000149011612 2.297463357448578
0.05000000074505806 2.441033184528351
0.20000000298023224 1.9897767305374146
0.05000000074505806 2.787104606628418
0.05000000074505806 2.3430312871932983
0.07500000298023224 2.264428585767746
0.1750000063329935 2.434328407049179
0.05000000074505806 2.482831448316574
0.02500000037252903 2.252169042825699
0.17500000447034836 2.4704861640930176
0.1250000037252903 2.13441693782

0.05000000074505806 2.286033272743225
0.1750000063329935 2.313494384288788
0.15000000409781933 2.3032931089401245
0.1250000037252903 2.4739153385162354
epoch 180, task:1, metrics: {'accuracy': 0.16666666915019354, 'loss': 1.8094984789689381}
epoch 180, task:2, metrics: {'accuracy': 0.10000000149011612, 'loss': 1.8086420794328053}
epoch 180, task:3, metrics: {'accuracy': 0.23333333929379782, 'loss': 1.821685512860616}
epoch 180, task:4, metrics: {'accuracy': 0.30000000943740207, 'loss': 1.8179561694463093}
epoch 180, task:5, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.828277478615443}
epoch 180, task:6, metrics: {'accuracy': 0.10000000149011612, 'loss': 1.8327156205972035}
================== TASK 7 / 86 =================
0.12500000186264515 2.211007833480835
0.05000000074505806 2.339182674884796
0.07500000298023224 1.8321216106414795
0.12500000186264515 2.218532979488373
0.15000000596046448 2.3739041686058044
0.12500000186264515 2.3259425163269043
0.12500000186264515 2.39713892

epoch 300, task:10, metrics: {'accuracy': 0.13333333532015482, 'loss': 1.7480981747309368}
================== TASK 11 / 86 =================
0.22500000707805157 2.273286372423172
0.15000000596046448 2.1825341284275055
0.02500000037252903 2.116659939289093
0.17500000447034836 2.327283054590225
0.05000000074505806 2.2517881393432617
0.07500000111758709 2.2745257318019867
0.15000000596046448 2.2053281664848328
0.1250000037252903 2.2326776683330536
0.05000000074505806 2.1371938586235046
0.0 2.2934202551841736
0.1250000037252903 2.441234737634659
0.02500000037252903 2.203367680311203
0.07500000111758709 2.10183385014534
0.1250000037252903 2.546042114496231
0.10000000149011612 2.278045356273651
0.1250000037252903 2.610551744699478
0.05000000074505806 2.151996076107025
0.22500000335276127 2.084664821624756
0.1250000037252903 2.3711535334587097
0.02500000037252903 2.162549316883087
0.10000000335276127 2.465970426797867
0.17500000447034836 2.265706568956375
0.12500000186264515 2.12135973572731


epoch 420, task:5, metrics: {'accuracy': 0.23333333681027094, 'loss': 1.7658655842145283}
epoch 420, task:6, metrics: {'accuracy': 0.16666666915019354, 'loss': 1.7636958758036296}
epoch 420, task:7, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.7584988574186962}
epoch 420, task:8, metrics: {'accuracy': 0.06666666766007741, 'loss': 1.7622845768928528}
epoch 420, task:9, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.7515613436698914}
epoch 420, task:10, metrics: {'accuracy': 0.16666666915019354, 'loss': 1.7402448654174805}
epoch 420, task:11, metrics: {'accuracy': 0.3333333407839139, 'loss': 1.764155904452006}
epoch 420, task:12, metrics: {'accuracy': 0.16666666915019354, 'loss': 1.7625176906585693}
epoch 420, task:13, metrics: {'accuracy': 0.16666666915019354, 'loss': 1.76303364833196}
epoch 420, task:14, metrics: {'accuracy': 0.10000000149011612, 'loss': 1.7381667892138164}
================== TASK 15 / 86 =================
0.1250000037252903 2.4540425539016724
0.225000003

epoch 510, task:14, metrics: {'accuracy': 0.06666666766007741, 'loss': 1.7301819721857707}
epoch 510, task:15, metrics: {'accuracy': 0.1666666716337204, 'loss': 1.7553964058558147}
epoch 510, task:16, metrics: {'accuracy': 0.2000000054637591, 'loss': 1.759030858675639}
epoch 510, task:17, metrics: {'accuracy': 0.30000000695387524, 'loss': 1.7596776286760967}
================== TASK 18 / 86 =================
0.17500000447034836 2.28080353140831
0.12500000186264515 2.1193958520889282
0.05000000074505806 2.2280180752277374
0.07500000111758709 2.3745491802692413
0.10000000149011612 2.281191885471344
0.15000000223517418 2.154133975505829
0.17500000447034836 2.158084213733673
0.05000000074505806 2.335695594549179
0.12500000186264515 2.3588118851184845
0.07500000111758709 2.457067549228668
0.15000000409781933 2.2853411436080933
0.15000000223517418 2.1071416437625885
0.0 2.1827177703380585
0.07500000298023224 2.2652406990528107
0.05000000074505806 2.210500478744507
0.07500000298023224 2.142932

...

epoch 2460, task:62, metrics: {'accuracy': 0.13333333532015482, 'loss': 1.7620413303375244}
epoch 2460, task:63, metrics: {'accuracy': 0.23333333929379782, 'loss': 1.7586103081703186}
epoch 2460, task:64, metrics: {'accuracy': 0.16666666915019354, 'loss': 1.7633461753527324}
epoch 2460, task:65, metrics: {'accuracy': 0.13333333532015482, 'loss': 1.7651970386505127}
epoch 2460, task:66, metrics: {'accuracy': 0.13333333532015482, 'loss': 1.7627024451891582}
epoch 2460, task:67, metrics: {'accuracy': 0.16666666915019354, 'loss': 1.739792803923289}
epoch 2460, task:68, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.766224483648936}
epoch 2460, task:69, metrics: {'accuracy': 0.26666667064030963, 'loss': 1.7638541261355083}
epoch 2460, task:70, metrics: {'accuracy': 0.10000000149011612, 'loss': 1.763159970442454}
epoch 2460, task:71, metrics: {'accuracy': 0.23333333929379782, 'loss': 1.758814811706543}
epoch 2460, task:72, metrics: {'accuracy': 0.23333333929379782, 'loss': 1.7542930940

epoch 2490, task:57, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.7542739709218342}
epoch 2490, task:58, metrics: {'accuracy': 0.26666667064030963, 'loss': 1.756695032119751}
epoch 2490, task:59, metrics: {'accuracy': 0.1666666716337204, 'loss': 1.7633461157480876}
epoch 2490, task:60, metrics: {'accuracy': 0.26666667064030963, 'loss': 1.748273362716039}
epoch 2490, task:61, metrics: {'accuracy': 0.2000000054637591, 'loss': 1.7640048662821453}
epoch 2490, task:62, metrics: {'accuracy': 0.23333333929379782, 'loss': 1.7618127465248108}
epoch 2490, task:63, metrics: {'accuracy': 0.13333333532015482, 'loss': 1.7581969300905864}
epoch 2490, task:64, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.7663578589757283}
epoch 2490, task:65, metrics: {'accuracy': 0.10000000149011612, 'loss': 1.7618055939674377}
epoch 2490, task:66, metrics: {'accuracy': 0.23333333681027094, 'loss': 1.7667187849680583}
epoch 2490, task:67, metrics: {'accuracy': 0.10000000149011612, 'loss': 1.7646253307

epoch 2520, task:51, metrics: {'accuracy': 0.16666666915019354, 'loss': 1.756636917591095}
epoch 2520, task:52, metrics: {'accuracy': 0.16666666915019354, 'loss': 1.7528932094573975}
epoch 2520, task:53, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.7626951336860657}
epoch 2520, task:54, metrics: {'accuracy': 0.1666666716337204, 'loss': 1.760413904984792}
epoch 2520, task:55, metrics: {'accuracy': 0.30000000695387524, 'loss': 1.7617207566897075}
epoch 2520, task:56, metrics: {'accuracy': 0.23333333681027094, 'loss': 1.762821873029073}
epoch 2520, task:57, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.749985655148824}
epoch 2520, task:58, metrics: {'accuracy': 0.13333333532015482, 'loss': 1.7596454222997029}
epoch 2520, task:59, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.7666348616282146}
epoch 2520, task:60, metrics: {'accuracy': 0.26666667064030963, 'loss': 1.750066727399826}
epoch 2520, task:61, metrics: {'accuracy': 0.10000000149011612, 'loss': 1.764419058958

epoch 2550, task:45, metrics: {'accuracy': 0.2666666731238365, 'loss': 1.7519588669141133}
epoch 2550, task:46, metrics: {'accuracy': 0.13333333532015482, 'loss': 1.7636384169260662}
epoch 2550, task:47, metrics: {'accuracy': 0.23333333681027094, 'loss': 1.764880100886027}
epoch 2550, task:48, metrics: {'accuracy': 0.30000000695387524, 'loss': 1.760649045308431}
epoch 2550, task:49, metrics: {'accuracy': 0.30000000943740207, 'loss': 1.766472081343333}
epoch 2550, task:50, metrics: {'accuracy': 0.16666666915019354, 'loss': 1.753519594669342}
epoch 2550, task:51, metrics: {'accuracy': 0.23333333681027094, 'loss': 1.7577905257542927}
epoch 2550, task:52, metrics: {'accuracy': 0.23333333929379782, 'loss': 1.7525077263514202}
epoch 2550, task:53, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.7627222935358684}
epoch 2550, task:54, metrics: {'accuracy': 0.10000000149011612, 'loss': 1.7629491686820984}
epoch 2550, task:55, metrics: {'accuracy': 0.2666666731238365, 'loss': 1.765844980875

epoch 2580, task:38, metrics: {'accuracy': 0.16666666915019354, 'loss': 1.7600558201471965}
epoch 2580, task:39, metrics: {'accuracy': 0.13333333532015482, 'loss': 1.7617101271947224}
epoch 2580, task:40, metrics: {'accuracy': 0.23333333681027094, 'loss': 1.752754012743632}
epoch 2580, task:41, metrics: {'accuracy': 0.2666666731238365, 'loss': 1.7599574724833171}
epoch 2580, task:42, metrics: {'accuracy': 0.23333333681027094, 'loss': 1.762721598148346}
epoch 2580, task:43, metrics: {'accuracy': 0.23333333929379782, 'loss': 1.7715450326601665}
epoch 2580, task:44, metrics: {'accuracy': 0.23333333681027094, 'loss': 1.7600255012512207}
epoch 2580, task:45, metrics: {'accuracy': 0.2666666731238365, 'loss': 1.7639466126759846}
epoch 2580, task:46, metrics: {'accuracy': 0.30000000695387524, 'loss': 1.7592100103696187}
epoch 2580, task:47, metrics: {'accuracy': 0.16666666915019354, 'loss': 1.758840004603068}
epoch 2580, task:48, metrics: {'accuracy': 0.2000000054637591, 'loss': 1.754870076974